Use this utlity to update the returns and std_dev fields within investment-options.csv  
To add more seurities, manually edit investment-options-in.csv or use the qgrid table in this notebook.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import csv
from datetime import datetime
from shutil import copyfile

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qgrid

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
filename = 'investment-options-in.csv'

In [6]:
# Read in investment options input csv.
header = None
with open('investment-options-header.csv', 'r') as f:
    header = f.read() + '\n'
df = pd.read_csv(filename, skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])

In [7]:
# Get list of all asset classes.  This will be used to populate the 'Asset Class' column
# with all possible choices.
filepath = bb.ROOT / 'universe' / 'asset-class-galaxy' / 'asset-classes.csv'
asset_classes = pd.read_csv(filepath, skip_blank_lines=True, comment='#')
asset_classes = list(asset_classes['Asset Class A'])
asset_classes = set(asset_classes)
asset_classes = sorted(asset_classes)

In [8]:
# Make qgrid widget to allow editing of 
df['Asset Class'] = pd.Categorical(df['Asset Class'], asset_classes)
column_definitions = {
    'Investment Option': {
        'width': 800,
    },
    'Description': {
        'width': 800,
    },
    'Asset Class': {
        # SlickGrid column options.
        'defaultSortAsc': True,
        'maxWidth': None,
        'minWidth': 50,
        'resizable': True,
        'sortable': True,
        'toolTip': "",
        'width': 800,
        # qgrid column options.
        'editable': True,
    }
}

qgrid_widget = qgrid.show_grid(df, show_toolbar=True, column_definitions=column_definitions)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [9]:
# Update df with user changes.
df = qgrid_widget.get_changed_df()

In [10]:
# Create save button.
def on_button_clicked(_):
    # backup current file
    timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    backup = filename + '-' + timestamp
    copyfile(filename, backup)
    
    # Save changes.
    with open(filename, 'w') as f:
        f.write(header)
    df.to_csv(filename, mode='a', index=False, header=False, quoting=csv.QUOTE_ALL)
    print('Saved changes to "{}"'.format(filename))

# Creates a button that allows us to save our changes.
button = widgets.Button(description='SAVE')
out = widgets.Output()
    
# Linking button and function together using a button's method.
button.on_click(on_button_clicked)

# Displaying button and its output together.
widgets.VBox([button,out])

In [11]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [12]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

......


In [13]:
# Read symbols timeseries into a dataframe.
ts = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
ts.set_index("Date", inplace=True)

In [14]:
annual_returns = bb.annualize_returns(ts, timeperiod='daily', years=1)

In [15]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualize_returns(ts, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(ts, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(ts, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(ts, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(ts, timeperiod='daily', years=5)

In [16]:
# Calculate 20 day annualized volatility.
daily_returns = ts.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [17]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [18]:
# Resample ts on a monthly basis.
ts.index = pd.to_datetime(ts.index)
monthly = ts.resample('M').ffill()

In [19]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [20]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [21]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [22]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [23]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')